Bibliotecas importantes

In [1]:
import pandas as pd
import re
import plotly.express as px

# Tratamento para "Centros_de_pesquisa_clínica_ativos.csv"

Entendimento inicial

In [41]:
# Leitura dos dados
path = "Centros_de_pesquisa_clínica_ativos.csv"
df_centros_de_pesquisa = pd.read_csv(path, sep=',', encoding='utf-8')
df_centros_de_pesquisa_novo = df_centros_de_pesquisa.copy()

In [42]:
print("Formato do dataset:", df_centros_de_pesquisa.shape)
print("\nColunas disponíveis:\n", df_centros_de_pesquisa.columns.tolist())
print("\nTipos de dados:\n", df_centros_de_pesquisa.dtypes)
print("\nValores ausentes:\n", df_centros_de_pesquisa.isna().sum())

Formato do dataset: (971, 3)

Colunas disponíveis:
 ['NCTId', 'Sponsor', 'Locations']

Tipos de dados:
 NCTId        object
Sponsor      object
Locations    object
dtype: object

Valores ausentes:
 NCTId        0
Sponsor      0
Locations    0
dtype: int64


In [43]:
# Pequena amostra dos dados
df_centros_de_pesquisa.head()

,NCTId,Sponsor,Locations
0,NCT01495533,"University Hospital, Saarland","Danta Pazzanese Heart Institute, São Paulo, Br..."
1,NCT02893852,Federal University of Minas Gerais,"Federal University of Minas Gerais, Belo Horiz..."
2,NCT00325676,Takeda,"Altana Pharma/Nycomed, Belo Horizonte-MG, Braz..."
3,NCT05864417,Johnson & Johnson Consumer Inc. (J&JCI),"Allergisa Pesquisa Dermato-Cosmética Ltda, Cam..."
4,NCT03190850,Hospital de Clinicas de Porto Alegre,"Hospital de Clinicas de Porto Alegre, Porto Al..."


Tratamento

In [ ]:
# Separando a coluna 'Locations' em várias linhas separadas por ponto e vírgula
# Exemplo de valor: "Prof. Dr. Emilio Pizzichini, Florianópolis, Brazil; Prof. Dr. Alberto Cukier, São Paulo, Brazil; Prof. Dr. Rafael Stelmach, São Paulo, Brazil"
# Deve ficar:
# "Prof. Dr. Emilio Pizzichini, Florianópolis, Brazil"
# "Prof. Dr. Alberto Cukier, São Paulo, Brazil"
# "Prof. Dr. Rafael Stelmach, São Paulo, Brazil"
df_centros_de_pesquisa_novo = df_centros_de_pesquisa_novo.assign(Locations=df_centros_de_pesquisa_novo['Locations'].str.split(';')).explode('Locations').reset_index(drop=True)



# Separando a coluna 'Locations' em 'Place' e 'City/state', eliminado o país, mas separando apenas as duas últimas vírgulas
df_centros_de_pesquisa_novo[['Place', 'City/state']] = df_centros_de_pesquisa_novo['Locations'].str.rsplit(',', n=2, expand=True)[[0, 1]].apply(lambda x: x.str.strip())

df_centros_de_pesquisa_novo.drop(columns=['Locations'], inplace=True)

# Devemos também assumir que locais com 'City/state' como 'None' são do mesmo estado que alguma outra instancia do mesmo 'NCTId'
# df_centros_de_pesquisa_novo['City/state'] = df_centros_de_pesquisa_novo.groupby('NCTId')['City/state'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

# Ou dropar linhas com 'City/state' como 'None'
# df_centros_de_pesquisa_novo = df_centros_de_pesquisa_novo[df_centros_de_pesquisa_novo['City/state'].notna()]


In [51]:
df_centros_de_pesquisa_novo

,NCTId,Sponsor,Place,City/state
0,NCT01495533,"University Hospital, Saarland",Danta Pazzanese Heart Institute,São Paulo
1,NCT02893852,Federal University of Minas Gerais,Federal University of Minas Gerais,Belo Horizonte
2,NCT00325676,Takeda,Altana Pharma/Nycomed,Belo Horizonte-MG
3,NCT00325676,Takeda,Altana Pharma/Nycomed,Blumenau - SC
4,NCT00325676,Takeda,Altana Pharma/Nycomed,Botucatu - SP
...,...,...,...,...
3261,NCT05639478,University of Sao Paulo General Hospital,Equipe Multidisciplinar E-Multi,Limeira
3262,NCT02259543,University of Sao Paulo,School of Dentistry at Bauru- University of Sã...,None
3263,NCT02259543,University of Sao Paulo,Clinics of Periodontics,Bauru
3264,NCT02074917,University of Sao Paulo,Instituto de Ortopedia e Traumatologia do HCFMUSP,São Paulo


In [ ]:
df_centros_de_pesquisa_novo.to_csv('../dados_tratados/Centros_de_pesquisa_cinica_ativos_tratado.csv', sep=';', index=False)